# Exploring the OpenAIRE Research Graph

In this notebook we explore how to integrate "our" resources into the OpenAIRE Research Graph. 

For more information …

* on the OpenAIRE LOD Service see http://lod.openaire.eu/documentation 
* on the OpenAIRE Research Graph [Schema documentation](http://lod.openaire.eu/vocab)
* or the [official document](https://doi.org/10.5281/zenodo.2643199) at Zenodo

A more human-friendly user interface is available at https://explore.openaire.eu/

Note that for some reason errors from the SPARQL endpoint are not passed correctly to the notebook. In case you need to debug a query, try to run it at http://lod.openaire.eu/endpoint in order to see the triple store's response.

## Questions

* Wo liegt welche Ressource ?
* Wie hängen Projekt & Datenset zusammen? (Im Modell? In den tatsächlichen Daten?)
* Lassen sich FWF-Projekte in ARCHE und OpenAIRE mappen?

## TODOS: 

* match against ARCHE RDF Dump
* match against Omnipot
* match Project List via FWF Project IDs

First, we need to configure to use the OpenAIRE Sparql endpoint:

In [126]:
import pandas as pd
import ipywidgets as widgets
from math import ceil
import re
from IPython.display import display
pd.set_option('display.max_colwidth', 4000)
pd.set_option('display.max_rows',1000)

from IPython.core.display import display, HTML
# get the notebook more screen estate
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
%%graph_notebook_config
{
  "host": "lod.openaire.eu",
  "port": "",
  "ssl": false,
  "sparql": {
    "path": "sparql"
  }
}


set notebook config to:
{
  "host": "lod.openaire.eu",
  "port": "",
  "ssl": false,
  "sparql": {
    "path": "sparql"
  },
  "gremlin": {
    "traversal_source": "g"
  }
}


## Help

In [3]:
%%sparql --help

SELECT *
WHERE {
    ?s ?p ?o
}

usage: ipykernel_launcher.py [-h] [--path PATH] [--expand-all] [--explain-type {dynamic,static,details}] [--explain-format {text/csv,text/html}] [-g GROUP_BY] [-gr] [-d DISPLAY_PROPERTY] [-de EDGE_DISPLAY_PROPERTY] [-t TOOLTIP_PROPERTY]
                             [-te EDGE_TOOLTIP_PROPERTY] [-l LABEL_MAX_LENGTH] [-le EDGE_LABEL_MAX_LENGTH] [--store-to STORE_TO] [--ignore-groups] [-sp] [-sd SIMULATION_DURATION] [--silent] [-r RESULTS_PER_PAGE] [--no-scroll]
                             [query_mode]

positional arguments:
  query_mode            query mode (default=query) [query|explain]

options:
  -h, --help            show this help message and exit
  --path PATH, -p PATH  prefix path to sparql endpoint. For example, if "foo/bar" were specified, the endpoint called would be host:port/foo/bar
  --expand-all
  --explain-type {dynamic,static,details}
                        explain mode to use when using the explain query mode
  --explain-format {text/csv,text/html}
                   

SystemExit: 0

/home/dschopper/data/DBA/acdh-dashboard-notebooks/acdh-dashboard-notebooks/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Getting a general overivew: 

We start by listing classes, ordered by their frequency.

In [4]:
%%sparql -r 20

SELECT IRI(?type) AS ?iri, COUNT(?s) AS ?cnt
WHERE {
    ?s a ?type.
}
GROUP BY ?type 
ORDER BY DESC(?cnt)
LIMIT 100

Similarly, we expolore the most used predicates.

WATCHME This takes quite some time / resources!

In [5]:
%%sparql

SELECT IRI(?p) AS ?iri, COUNT(?p) AS ?cnt
WHERE {
    ?s ?p ?o.
}
GROUP BY ?p 
ORDER BY DESC(?cnt)
LIMIT 100

Listing all values of the `oav:country` property, we find that Austria is either represented by string "AT" or by URI `<http://dbpedia.org/resource/Austria>` 

In [6]:
%%sparql

PREFIX oav: <http://lod.openaire.eu/vocab/>

SELECT ?country, COUNT(?subject) AS ?cnt
WHERE { ?subject oav:country ?country }
GROUP BY ?country
ORDER BY ASC(?country)

Next, we analyze which nodes relate to Austria, and group them by class. 
It seems that the two possible propery values of `oav:country` correlate with the subject class: 
Instances of `ResultEntity` have "AT" whereas instances of `OrganizationEntity` use `<http://dbpedia.org/resource/Austria>` 

In [7]:
%%sparql

PREFIX oav: <http://lod.openaire.eu/vocab/>

SELECT ?type, ?country, COUNT(?subject) as ?mentioningInstances
WHERE {
    {?subject oav:country <http://dbpedia.org/resource/Austria>} UNION {?subject oav:country "AT"}.
    ?subject a  ?type
}
GROUP BY ?type ?country
LIMIT 100

We now can further investigate both classes, `ResultEntity` and `OrganizationEntity`:

### Exploring organisations related to Austria 



We want to get to know a bit better the instances of `OrganizationEntity`related to Austria, so we get a list of their property classes:

In [8]:
%%sparql

PREFIX oav: <http://lod.openaire.eu/vocab/>
PREFIX dbp: <http://dbpedia.org/resource/>

SELECT DISTINCT(?p)
WHERE {
    ?org  oav:country dbp:Austria;
          a           oav:OrganizationEntity;
          ?p          ?o.
}
LIMIT 100

Initially, we want to get the names of those organizations. In order not to overwhelm the notebook by rendering a table with so many entries we use `--silent` and store the results to a variable instead.

In [15]:
%%sparql --store-to orgsResults --silent

PREFIX oav: <http://lod.openaire.eu/vocab/>
PREFIX dbp: <http://dbpedia.org/resource/>

SELECT ?org, GROUP_CONCAT(?orgName, "; ") AS ?names
WHERE {
    ?org  oav:country dbp:Austria;
          a           oav:OrganizationEntity;
          oav:legalname ?orgName
}
GROUP BY ?org

Then, we can load the results into a DataFrame for further exploration.

In [16]:
orgTuples=[]

for o in orgsResults["results"]["bindings"]:
    orgTuples.append((o["org"]["value"], o["names"]["value"]))
orgs = pd.DataFrame(orgTuples, columns=["ID", "Name(s)"])

if orgsResults:
    del orgsResults

In [120]:
print(len(orgs))
orgs.sort_values("Name(s)").head(25)

4306


,ID,Name(s)
3647,http://lod.openaire.eu/data/organization/fwf_________::050ab65cd761d189d38ca9b1d3fdf725,- Deutsches Archäologisches Institut
2669,http://lod.openaire.eu/data/organization/fwf_________::1f955b52fb727c7e242187d2dd0718c1,"- Privat, Burgenland"
1189,http://lod.openaire.eu/data/organization/fwf_________::c114c0731dcd58c7cd81bd065538d0d9,"- Privat, NÖ"
742,http://lod.openaire.eu/data/organization/fwf_________::6f445043a6e7c709b9e3b22f4081b89e,"- Privat, OÖ"
1914,http://lod.openaire.eu/data/organization/fwf_________::41b083dbf8489005dbe4024d59976fa1,"- Privat, Salzburg"
2547,http://lod.openaire.eu/data/organization/fwf_________::f4a215516eaa52a5c3f8d85be8df9065,"- Privat, Salzburg; Österreichische Akademie der Wissenschaften - Institut für kunst- und musikhistorische Forschungen"
2549,http://lod.openaire.eu/data/organization/fwf_________::5d71561ffe247b94977b73567d40122e,"- Privat, Steiermark"
2916,http://lod.openaire.eu/data/organization/fwf_________::11d6ba3e90d9ea64b896be8af5bfd393,"- Privat, Steiermark; Universität Graz - Institut für Germanistik"
1739,http://lod.openaire.eu/data/organization/fwf_________::b3278ac9505746b767b0ef4c160e2b9e,"- Privat, Tirol"
862,http://lod.openaire.eu/data/organization/fwf_________::ff89b44e580bb5566994d4c0082b0e40,"- Privat, Tirol; Universität Innsbruck - Institut für Angewandte Physik"


This cell is not used – it contains code blocks to build a more-user-friendly filter widget

```
def pageThroughDf(df, sortColumn, pageParam = 1, pageSize = 50):
    sliceStart = (pageParam * pageSize) - (pageSize-1)
    sliceEnd = pageParam * pageSize
    return df.sort_values(sortColumn).iloc[sliceStart:sliceEnd]


filterInputField = widgets.Text(
    value='',
    placeholder='Type filter text',
    description='filter by Name:',
    disabled=False
)

def filterInputSubmission(param):
    output.clear_output()    
    return orgs
    filterDf(orgs,"Name(s)", param.value, 1, pageSize=50)
    #for i in filterDf(orgs,"Name(s)", param.value, 1, pageSize=50):
    #    output.append_display_data(i)
    
#output = widgets.Output()
#display(filterInputField, output)
#filterInputField.on_submit(filterInputSubmission)
```


Next, we try to identify ÖAW / ACDH-CH and its parts / predecessors in the data by a list of name candidates:

In [110]:
nameCandidates = [
    "ACDH", 
    "for Digital Humanities", 
#    "Akademie der Wissenschaften", 
#    "Austrian Academy of Sciences",
#    "ÖAW", 
    "Institut für Österreichische Dialekt- und Namenslexika", 
    "DINAMLEX", 
    "Texttechnologie und Korpuslinguistik",
    "ICLTT",
    "AAC",
    "Acadmic Corpora",
    " ACE ",
    "ÖBL", 
    "Österreichisches Biographisches Lexikon",
    "Institut für kunst- und musikhistorische Forschungen"
]

We compile a regex expression from this list, filter the orgs dataframe by it and attach the matching strings into a new column, leading to a new variable `acdhOrgs`.

In [122]:
nameCandidatesEscaped = []
for n in nameCandidates:
    nameCandidatesEscaped.append(re.escape(n))
regex = '('+'|'.join(nameCandidatesEscaped)+')'
print(regex)
#orgs['Name(s)'][]

matchingOrgs = orgs[orgs['Name(s)'].str.contains(regex, regex=True)]
acdhOrgs = matchingOrgs.join(matchingOrgs["Name(s)"].str.extract(regex)).sort_values(0)

acdhOrgs

(ACDH|for\ Digital\ Humanities|Institut\ für\ Österreichische\ Dialekt\-\ und\ Namenslexika|DINAMLEX|Texttechnologie\ und\ Korpuslinguistik|ICLTT|AAC|Acadmic\ Corpora|\ ACE\ |ÖBL|Österreichisches\ Biographisches\ Lexikon|Institut\ für\ kunst\-\ und\ musikhistorische\ Forschungen)


<ipython-input-122-c2456ecc1ac2>:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  matchingOrgs = orgs[orgs['Name(s)'].str.contains(regex, regex=True)]


,ID,Name(s),0
168,http://lod.openaire.eu/data/organization/fwf_________::712f9932ffd84730733083b3f3642fc5,Österreichische Akademie der Wissenschaften - ACE - Austrian Corpora and Editions,ACE
2547,http://lod.openaire.eu/data/organization/fwf_________::f4a215516eaa52a5c3f8d85be8df9065,"- Privat, Salzburg; Österreichische Akademie der Wissenschaften - Institut für kunst- und musikhistorische Forschungen",Institut für kunst- und musikhistorische Forschungen
1305,http://lod.openaire.eu/data/organization/fwf_________::36a9eac4dff9f4f7d918478065aca8c7,Universität Klagenfurt - Institut für Wirtschaftsinformatik und Anwendungssysteme IWAS; Österreichische Akademie der Wissenschaften - Institut für Österreichische Dialekt- und Namenslexika; Technische Universität Graz - Institut für GEOinformation,Institut für Österreichische Dialekt- und Namenslexika
2224,http://lod.openaire.eu/data/organization/fwf_________::0850498021529d3e4a73aaa07087c3bc,Alpen-Adria-Universität Klagenfurt - Institut für Wirtschaftsinformatik und Anwendungssysteme IWAS; Österreichische Akademie der Wissenschaften - Institut für Österreichische Dialekt- und Namenslexika; Technische Universität Graz - Institut für GEOinfor,Institut für Österreichische Dialekt- und Namenslexika
119,http://lod.openaire.eu/data/organization/opendoar____::f52deebaec619a64253a732864b5e23d,"Austrian Centre for Digital Humanities, University of Graz",for Digital Humanities
3084,http://lod.openaire.eu/data/organization/fwf_________::74d24e98df16045624b1371cec8ce73e,Österreichische Akademie der Wissenschaften - Austrian Centre for Digital Humanities (ACDH),for Digital Humanities
2787,http://lod.openaire.eu/data/organization/fwf_________::3774596385adcd380f4254b1e8a861ff,Universität Wien - Institut für Anglistik und Amerikanistik; Österreichische Akademie der Wissenschaften - Austrian Centre for Digital Humanities (ACDH),for Digital Humanities
2377,http://lod.openaire.eu/data/organization/fwf_________::fbb0163d66fe6deffd7ea2c98796ccc0,Universität Wien - Institut für Sprachwissenschaft; Österreichische Akademie der Wissenschaften - Austrian Centre for Digital Humanities (ACDH),for Digital Humanities
1795,http://lod.openaire.eu/data/organization/fwf_________::27d5e510117e26749fd13c258a41964d,Universität Innsbruck - Institut für Geschichtswissenschaften und Europäische Ethnologie; Österreichische Akademie der Wissenschaften - Austrian Centre for Digital Humanities (ACDH),for Digital Humanities
1676,http://lod.openaire.eu/data/organization/fwf_________::408a03e0362307292fc76f87972b8e92,Österreichische Akademie der Wissenschaften - Austrian Centre for Digital Humanities (ACDH); Österreichische Akademie der Wissenschaften - Institut für Schallforschung; Österreichische Akademie der Wissenschaften - Institut für Corpuslinguistik und Text,for Digital Humanities


In [123]:
len(acdhOrgs)
acdhOrgs.groupby(by=0).size()

0
 ACE                                                       1
Institut für kunst- und musikhistorische Forschungen       1
Institut für Österreichische Dialekt- und Namenslexika     2
for Digital Humanities                                    12
Österreichisches Biographisches Lexikon                    1
dtype: int64

In [411]:
n="Institut für Österreichische Dialekt- und Namenslexika"
f = filterDf(orgs,"Name(s)", n).sort_values("Name(s)")
print(str(len(f))+ " occurences for '"+n+"'")
f

2 occurences for 'Institut für Österreichische Dialekt- und Namenslexika'


,ID,Name(s)
2224,http://lod.openaire.eu/data/organization/fwf_________::0850498021529d3e4a73aaa07087c3bc,Alpen-Adria-Universität Klagenfurt - Institut für Wirtschaftsinformatik und Anwendungssysteme IWAS; Österreichische Akademie der Wissenschaften - Institut für Österreichische Dialekt- und Namenslexika; Technische Universität Graz - Institut für GEOinfor
1305,http://lod.openaire.eu/data/organization/fwf_________::36a9eac4dff9f4f7d918478065aca8c7,Universität Klagenfurt - Institut für Wirtschaftsinformatik und Anwendungssysteme IWAS; Österreichische Akademie der Wissenschaften - Institut für Österreichische Dialekt- und Namenslexika; Technische Universität Graz - Institut für GEOinformation


In [115]:
n=" ACE "
f = filterDf(orgs,"Name(s)", n).sort_values("Name(s)")
print(str(len(f))+ " occurences for '"+n+"'")
f

1 occurences for ' ACE '


,ID,Name(s)
168,http://lod.openaire.eu/data/organization/fwf_________::712f9932ffd84730733083b3f3642fc5,Österreichische Akademie der Wissenschaften - ACE - Austrian Corpora and Editions


### Organizations: Properties and Outgoing references

Based on the list of properties above, we try to get the full overview of all relevant instances:

In [141]:
%%sparql

PREFIX oav: <http://lod.openaire.eu/vocab/>
PREFIX dbp: <http://dbpedia.org/resource/>

SELECT * 
WHERE {
    ?org  oav:country dbp:Austria;
          a           oav:OrganizationEntity;
          oav:legalname ?orgName.
          
    OPTIONAL { ?org oav:orignalid ?orginalid. }
    OPTIONAL { ?org oav:pid ?pid. }
    OPTIONAL { ?org owl:sameAs ?sameAs. }
    OPTIONAL { ?org oav:provision ?provision. }
    OPTIONAL { ?org oav:legalshortname ?legalshortname.}
    OPTIONAL { ?org oav:affiliation ?affiliation.}
    OPTIONAL { ?org oav:description ?descriptiond.}
    OPTIONAL { ?org oav:participation ?participation.}
    OPTIONAL { ?org oav:collectedfrom ?collectedfrom.}
    OPTIONAL { ?org oav:dateofcollection ?dateofcollection. }
    OPTIONAL { ?org oav:dateoftransformation ?dateoftransformation. }
    OPTIONAL { ?org oav:websiteurl ?websiteurl. }
    

    FILTER(CONTAINS(?orgName, "for Digital Humanities"))
}

LIMIT 100

### Organizations: incoming references

Finally, we analyse which incoming references to organizations exist:

In [283]:
%%sparql

PREFIX oav: <http://lod.openaire.eu/vocab/>
PREFIX dbp: <http://dbpedia.org/resource/>

SELECT GROUP_CONCAT(DISTINCT(?domainClass), "") AS ?domainClass, GROUP_CONCAT(DISTINCT(?property),"") AS ?propertyClass, COUNT(?domain) AS ?domainCnt
WHERE {
    ?org  oav:country dbp:Austria.
    ?org     a           oav:OrganizationEntity;
             oav:legalname ?orgName.
          
    ?source ?property ?org;
            a ?domainClass
    FILTER(CONTAINS(?orgName, "for Digital Humanities"))
}
GROUP BY ?property
LIMIT 100

Although not explained in the Schema, the semantics of `Project > participation > Organization` seems rather clear and will be analyzed below. However, before deep-diving into the project entity class, we want to further explore the `Datasource > provision > Organization` path:

### Conclusions on treatment of organizations

* Institutes are treated as organizations equally as their mother institutions – evidently without any indication of the difference or relation between both
* OpenDOAR and re3data seem to contribute "real" organizations while it seems that in the data submitted by FWF, every cooperation constellation in a project is represented as an `OrganizationEntity` – this is a massive problem given that we don't have one single identifier for a partner institution we are interested in.
* Organization metadata seems really scarse everywhere – no PIDs, sameAs, affiliation or description.
* Only oav:website might be useful for deduplication if needed

## Datasources

Let's see how a `DatasourceEntity` is described in the OpenAIRE Graph:

In [142]:
%%sparql

PREFIX oav: <http://lod.openaire.eu/vocab/>
PREFIX dbp: <http://dbpedia.org/resource/>

SELECT DISTINCT(?dsProperty)
WHERE {
    ?org  oav:country dbp:Austria.
    ?org     a           oav:OrganizationEntity;
             oav:legalname ?orgName.
          
    ?datasource oav:provision ?org;
                ?dsProperty ?dsPropertyValue
    FILTER(CONTAINS(?orgName, "for Digital Humanities"))
}
ORDER BY ?dsProperty
LIMIT 100

And then, how ARCHE is represented:

In [215]:
%%sparql --expand-all

PREFIX oav: <http://lod.openaire.eu/vocab/>
PREFIX dbp: <http://dbpedia.org/resource/>

SELECT 
    DISTINCT(?datasource), 
    GROUP_CONCAT(DISTINCT(?officialname),"") AS ?officialname, 
    GROUP_CONCAT(DISTINCT(?subjects), " // ") AS ?subjects,
    GROUP_CONCAT(DISTINCT(?englishname), "") AS ?englishname,
    GROUP_CONCAT(DISTINCT(?datasourcetype),"") AS ?datasourcetype,
    GROUP_CONCAT(DISTINCT(?description),"") AS ?description,
    GROUP_CONCAT(DISTINCT(?journal),"") AS ?journal,
    GROUP_CONCAT(DISTINCT(?namespaceprefix),"") AS ?namespaceprefix,
    GROUP_CONCAT(DISTINCT(?websiteurl),"") AS ?websiteurl,
    GROUP_CONCAT(DISTINCT(?versioning),"") AS ?versioning,
    GROUP_CONCAT(DISTINCT(?collectedfrom),"") AS ?collectedfrom,
    CONCAT(GROUP_CONCAT(DISTINCT(?latitude),"")," ", GROUP_CONCAT(DISTINCT(?longitude),"")) AS ?coords
WHERE {
    ?org  oav:country dbp:Austria.
    ?org     a           oav:OrganizationEntity;
             oav:legalname ?orgName.
          
    ?datasource oav:provision ?org.
    OPTIONAL { ?datasource oav:officialname ?officialname. }
    OPTIONAL { ?datasource  oav:englishname ?englishname.}
    OPTIONAL { ?datasource  oav:subjects ?subjects .}
    OPTIONAL { ?datasource oav:datasourcetype ?datasourcetype. }
    OPTIONAL { ?datasource oav:description  ?description. } 
    OPTIONAL { ?datasource oav:journal  ?journal. } 
    OPTIONAL { ?datasource oav:websiteurl  ?websiteurl. } 
    OPTIONAL { ?datasource oav:versioning  ?versioning. } 
    OPTIONAL { ?datasource oav:collectedfrom  ?collectedfrom. } 
    OPTIONAL { ?datasource oav:latitude  ?latitude. } 
    OPTIONAL { ?datasource oav:longitude  ?longitude. } 
    OPTIONAL { ?datasource oav:namespaceprefix  ?namespaceprefix. } 
    FILTER(CONTAINS(?orgName, "for Digital Humanities"))
}
GROUP BY ?datasource
LIMIT 100

There are no incoming relations to any `DatasourceEntity`.

In [245]:
%%sparql

PREFIX oav: <http://lod.openaire.eu/vocab/>
PREFIX dbp: <http://dbpedia.org/resource/>

SELECT *
WHERE {
    ?s ?p ?o.
    ?o a oav:DatasourceEntity
}
LIMIT 100

### Conclusions on Datasource

* Strangely enough, datasources have no incoming relations – not even datasets / resultEntities so it seems that we don't have a way of indicating dataset provenance directly.

## Projects

Next, we have a look at the properties of the `ProjectEntity` class.

In [257]:
%%sparql

PREFIX oav: <http://lod.openaire.eu/vocab/>
PREFIX dbp: <http://dbpedia.org/resource/>

SELECT ?p, COUNT(?p) AS ?cnt
WHERE {
  ?s ?p ?o;
      a oav:ProjectEntity
}
GROUP BY ?p
ORDER BY ?p ?cnt
LIMIT 100

As seen above, projects and organizations are linked with by `project oav:participation organization`. Let's use this to identify some projects affiliated with the ACDH-CH: 

In [282]:
%%sparql

PREFIX oav: <http://lod.openaire.eu/vocab/>
PREFIX dbp: <http://dbpedia.org/resource/>

SELECT ?objectType, COUNT(?objectType) AS ?objTypeCnt, ?subjectType, COUNT(?subjectType) AS ?subjTypeCnt
WHERE {
  ?s oav:participation ?p.
  ?s a ?subjectType.
  ?o a ?objectType
}
GROUP BY ?objectType ?subjectType
LIMIT 100

Keyboard interrupt detected.


In [281]:
%%sparql

PREFIX oav: <http://lod.openaire.eu/vocab/>
PREFIX dbp: <http://dbpedia.org/resource/>

SELECT DISTINCT(?title)
WHERE {
  ?project a oav:ProjectEntity;
      oav:title ?title;
      oav:participation ?org.
  ?org oav:legalname ?orgName.
  FILTER(CONTAINS(str(?orgName), "Centre for Digital"))    
}

LIMIT 100

## Datasets

In [272]:
%%sparql

PREFIX oav: <http://lod.openaire.eu/vocab/>

SELECT  *
WHERE {
  ?s    a    oav:ResultEntity;
        oav:resultType "dataset".
}
LIMIT 100